## Imports

In [47]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

In [1]:
import os
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from torchvision import models,transforms,datasets
import pandas as pd
from torch.autograd import Variable
import bcolz

In [2]:
# use_gpu = False
use_gpu = torch.cuda.is_available()
def gpu(x,use_gpu=use_gpu):
    if use_gpu:
        return x.cuda()
    else:
        return x

## Data Processing

In [3]:
data_dir = './Dataset'

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

vgg_format = transforms.Compose([
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

dsets = {x: ImageFolderWithPaths(os.path.join(data_dir, x), vgg_format)
         for x in ['train', 'val', 'test']}

    
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val','test']}
dset_classes = dsets['train'].classes
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=64,
                                               shuffle=True, num_workers=6)
                for x in ['train', 'val', 'test']}

In [4]:
c = 0
for inputs, labels, paths in dset_loaders['train']:
    # use the above variables freely
    print(inputs, labels, paths)
    c += 1
    if c == 1:
        break

tensor([[[[ 1.2557,  1.2899,  1.3755,  ...,  1.3242,  1.2214,  1.1529],
          [ 1.2557,  1.3070,  1.3755,  ...,  1.3413,  1.2385,  1.1700],
          [ 1.2214,  1.2728,  1.3242,  ...,  1.3584,  1.2728,  1.1872],
          ...,
          [ 0.3481,  0.3652,  0.3652,  ..., -1.3473, -1.3815, -1.3987],
          [ 0.2624,  0.2967,  0.3309,  ..., -1.3473, -1.3644, -1.3815],
          [ 0.1597,  0.2111,  0.2967,  ..., -1.3473, -1.3473, -1.3644]],

         [[ 0.9755,  1.0105,  1.0980,  ...,  1.5357,  1.4307,  1.3606],
          [ 0.9755,  1.0280,  1.0980,  ...,  1.5532,  1.4482,  1.3782],
          [ 0.9405,  0.9930,  1.0455,  ...,  1.5707,  1.4832,  1.3957],
          ...,
          [ 0.0126,  0.0301,  0.0476,  ..., -1.7556, -1.7731, -1.7906],
          [-0.0399, -0.0049,  0.0301,  ..., -1.7381, -1.7556, -1.7731],
          [-0.1099, -0.0574,  0.0301,  ..., -1.7381, -1.7381, -1.7556]],

         [[ 0.4788,  0.5136,  0.6008,  ...,  1.9080,  1.8034,  1.7337],
          [ 0.4788,  0.5311,  

In [5]:
# clean IngredientList.txt and output the doublets
ing_list = './IngredientList.txt'
text_file = open("./IngredientList.txt", "r")
lines = text_file.read().split('\n')
ing_dict = {}
i = 0
for l in lines:
    ing_dict[i] = l
    i += 1
for key, val in ing_dict.items():
    val = val.lower()
    if "\ufeff" in val:
        val = val.replace('\ufeff','')
    if "of" in val:
        val = val.replace('of ','')
    if "chiffonade" in val:
        val = val.replace('chiffonade','sliced')
    if "slices" in val:
        val = val.replace('slices','sliced')
    if 'julienned' in val:
        val = val.replace('julienned', 'shredded')
    val = val.replace(' ', '')
    ing_dict[key] = val

cut_set = {0: "batonnet", 1: "minced", 2: "sliced", 3: "crushed", 4: "shredded", 5: "brunoisediced", 6: "chopped", 7:"chunks", 8: "fried", 9: "steamed", 10: "boiled", 11: "pickled", 12: "seared", 13: "dried"}

new_ing_dict = {}
ing_set = {}
i, j = 0, 0

for idx, val in ing_dict.items():
    new_ing_dict[idx] = [0,-1]
    for jdx, cut in cut_set.items():
        if cut in val:
            new_ing_dict[idx][0] = jdx+1
            val = val.replace(cut, '')
    if not val in ing_set.values():
        ing_set[j] = val
        j += 1
        new_ing_dict[idx][1] = j
    else:
        new_ing_dict[idx][1] = list(ing_set.keys())[list(ing_set.values()).index(val)]



In [6]:
ing_set

{0: 'greenonion',
 1: 'whiteonion',
 2: 'ham',
 3: 'carrot',
 4: 'hobblockscarrot',
 5: 'blacksesame',
 6: 'whitesesame',
 7: 'pepper',
 8: 'wholegreenpepper',
 9: 'chineseparsleycoriander',
 10: 'beansprouts',
 11: 'bacon',
 12: 'tomatosclices',
 13: 'cherrytomato',
 14: 'greenvegetables',
 15: 'cornkernels',
 16: 'cornblocks',
 17: 'bittergourd',
 18: 'broccoli',
 19: 'cauliflower',
 20: 'toast',
 21: 'bread',
 22: 'chinesemahogany',
 23: 'crabsticks',
 24: 'bambooshootstips',
 25: 'chivepieces',
 26: 'chives',
 27: 'onion',
 28: 'pumpkinblocks',
 29: 'groundnutkernels',
 30: 'lemon',
 31: 'shelledfreshshrimps',
 32: 'seashrimp',
 33: 'freshshrimp',
 34: 'crayfish',
 35: 'cucumber',
 36: 'hobblockscucumber',
 37: 'hotanddrychili',
 38: 'hotanddrypepper',
 39: 'hotanddrypepperpowder',
 40: 'purplecabbage',
 41: 'chinesecabbage',
 42: 'blackfungus',
 43: 'perillacrispatanaka',
 44: 'ginger',
 45: 'scrambledegg',
 46: 'eggcake',
 47: 'eggcrepe',
 48: 'spicedcornedegg',
 49: 'eggyolk',
 

In [7]:
new_ing_dict

{0: [2, 1],
 1: [3, 0],
 2: [13, 0],
 3: [0, 2],
 4: [6, 3],
 5: [5, 2],
 6: [0, 2],
 7: [3, 2],
 8: [6, 4],
 9: [3, 3],
 10: [1, 3],
 11: [5, 3],
 12: [0, 5],
 13: [0, 6],
 14: [0, 7],
 15: [4, 8],
 16: [5, 7],
 17: [3, 7],
 18: [13, 7],
 19: [0, 7],
 20: [0, 9],
 21: [0, 10],
 22: [0, 11],
 23: [0, 12],
 24: [0, 13],
 25: [3, 14],
 26: [0, 13],
 27: [0, 15],
 28: [3, 14],
 29: [0, 16],
 30: [0, 17],
 31: [3, 18],
 32: [0, 19],
 33: [0, 20],
 34: [0, 21],
 35: [0, 22],
 36: [0, 23],
 37: [0, 24],
 38: [0, 25],
 39: [0, 26],
 40: [7, 27],
 41: [0, 26],
 42: [5, 28],
 43: [3, 27],
 44: [6, 27],
 45: [0, 29],
 46: [0, 30],
 47: [4, 29],
 48: [0, 31],
 49: [0, 32],
 50: [14, 33],
 51: [0, 34],
 52: [0, 35],
 53: [5, 36],
 54: [1, 35],
 55: [0, 37],
 56: [3, 35],
 57: [6, 35],
 58: [4, 38],
 59: [0, 39],
 60: [0, 40],
 61: [3, 41],
 62: [0, 42],
 63: [0, 43],
 64: [0, 44],
 65: [3, 45],
 66: [5, 44],
 67: [7, 44],
 68: [0, 46],
 69: [0, 47],
 70: [4, 48],
 71: [0, 49],
 72: [0, 50],
 73: [

In [8]:
ing_dict

{0: 'mincedgreenonion',
 1: 'slicedgreenonion',
 2: 'searedgreenonion',
 3: 'whiteonion',
 4: 'brunoisedicedham',
 5: 'shreddedham',
 6: 'ham',
 7: 'slicedham',
 8: 'brunoisedicedcarrot',
 9: 'slicedcarrot',
 10: 'batonnetcarrot',
 11: 'shreddedcarrot',
 12: 'hobblockscarrot',
 13: 'blacksesame',
 14: 'whitesesame',
 15: 'crushedpepper',
 16: 'shreddedpepper',
 17: 'peppersliced',
 18: 'searedpepper',
 19: 'pepper',
 20: 'wholegreenpepper',
 21: 'chineseparsleycoriander',
 22: 'beansprouts',
 23: 'bacon',
 24: 'tomatosclices',
 25: 'cherrytomatosliced',
 26: 'cherrytomato',
 27: 'greenvegetables',
 28: 'slicedgreenvegetables',
 29: 'cornkernels',
 30: 'cornblocks',
 31: 'bittergourdsliced',
 32: 'broccoli',
 33: 'cauliflower',
 34: 'toast',
 35: 'bread',
 36: 'chinesemahogany',
 37: 'crabsticks',
 38: 'bambooshootstips',
 39: 'chivepieces',
 40: 'choppedchives',
 41: 'chives',
 42: 'shreddedonion',
 43: 'onionsliced',
 44: 'brunoisedicedonion',
 45: 'pumpkinblocks',
 46: 'groundnutkern

## Model building

In [9]:
def norm(input, p=2, dim=1, eps=1e-12):
    return input / input.norm(p,dim,keepdim=True).clamp(min=eps).expand_as(input)

In [39]:
class embedding(nn.Module):
    def __init__(self):
        super(embedding, self).__init__()
        # new feature
        self.feature_emb = nn.Sequential(
            nn.Linear(512, 172), 
            nn.Tanh(),
        )
        # ingredient
        self.ingredient_emb = nn.Sequential(
            nn.Linear(172, 354),
            nn.Softmax(),
        )
         # cutting
        self.cutting_emb = nn.Sequential(
            nn.Linear(172, 14),
            nn.Softmax(),
        )
    
    def forward(self, x):
        x = x.view(-1, 512)
        fea_embedding = self.feature_emb(x)
        fea_embedding = norm(fea_embedding)
        ing_embedding = self.ingredient_emb(fea_embedding)
        cut_embedding = self.cutting_emb(fea_embedding)
        return fea_embedding, ing_embedding, cut_embedding 

In [30]:
class vggfeature(nn.Module):
    def __init__(self):
        super(vggfeature, self).__init__()
        # VGG16 model
        model_vgg = models.vgg16(pretrained = True)
        # pool5 feature maps
#         self.features = model_vgg.features
#         self.features = nn.Sequential(
#             *list(model_vgg.features.children())[:-1] ##
#         )
        self.features = model_vgg.features
    
        for p in self.parameters():
            p.requires_grad = False
        
#         self.pool5=nn.Sequential()
        # new feature
        
        self.feature_emb = nn.Sequential(
            nn.Linear(512, 172), 
            nn.Tanh(),
        )
        # ingredient
        self.ingredient_emb = nn.Sequential(
            nn.Linear(172, 354),
            nn.Softmax(),
        )
         # cutting
        self.cutting_emb = nn.Sequential(
            nn.Linear(172, 14),
            nn.Softmax(),
        )
    
    def forward(self, x):
        x = self.features(x)
        print(x.size)
        x = x.view(-1, 512)
        fea_embedding = self.feature_emb(x)
        fea_embedding = norm(fea_embedding)
        ing_embedding = self.ingredient_emb(fea_embedding)
        cut_embedding = self.cutting_emb(fea_embedding)
        return fea_embedding, ing_embedding, cut_embedding 
    

In [43]:
def idx_unflatten(tsor, dim):
    b = tsor.cpu().numpy()
    c = []
    for val in b:
        c.append([(int)(val/dim),val%dim])
    c = np.asarray(c)
    new_tensor = torch.from_numpy(c)
    return new_tensor

def pooling(ing_emb, cut_emb):
    
    m2 = ing_emb.size(0)
    ingDim=353
    # get the i-th region of ingredient j
    ing_tmp = ing_emb.view(-1, m2)    
    ing_max, idx = torch.max(ing_tmp, dim=1)
    ing_idx = idx_unflatten(idx, np.sqrt(m2))
    '''
    # select top-k ingredients
    ing, new_idx = torch.topk(ing_max, k)
    indices = torch.LongTensor(new_idx)
    ing_kidx = torch.index_select(ing_idx, 0, indices)
    '''
    # pooling cutting and cooking method of i-th region
    cut_max, cut_idx = torch.max(cut_emb, dim=0)   
    indices = ing_idx.long().cuda()
#     indices = Variable(ing_idx.long().cuda())
#     print(indices.type())
    cut_way = torch.index_select(cut_idx, 0, indices.view(-1))
    cut_proba = torch.index_select(cut_max, 0, indices.view(-1))
    
    # aggregate ingredient and cutting(resp.cooking) tensors
    p_ingre = torch.zeros([ingDim])
    for num, proba in enumerate(ing_max.detach().numpy()):
        p_ingre[num] = proba
    
    p_cut = torch.zeros([cutDim, ingDim])
    
    i = 0
    for idx, proba in zip(cut_way.numpy(), cut_proba.numpy()):
        p_cut[idx][i] = proba
        i += 1
   
    return p_ingre, p_cut

In [13]:
# original ingredient label is represented as a list
# output is a list of doublet(ing, cut)
def recipe_retrival(ingre_label):
    res = []
    for idx, val in enumerate(ingre_label):
        if val == 1:
            doublet = new_ing_dict[idx]             
            res.append(doublet)
    return res

IngreLabel = open('IngreLabel.txt', 'r').read().split('\n')[:-1]   # list of str
for i in range(len(IngreLabel)):
    IngreLabel[i] = IngreLabel[i].split()                    # list of list, element is str

# Get t_ingre from path
def Get_ingre(path):
    ID = path[11]
    if ID == 'r':    #train
        path = path[15:]
    elif ID == 'a':  #val
        path = path[13:]
    else:            #test
        path = path[14:]
    # path has the same format of that in IngreLabel.txt
    for pil in IngreLabel:
        if pil[0] == path:
            ingre = [int(i) for i in pil[1:]]
            return ingre

# Get t_cut from t_ingre
def Get_cut(t_ingre):
    t_cut = torch.zeros([cutDim, ingDim])
    recipe = recipe_retrival(t_ingre)
    a,b = zip(*recipe)
    c = list(a)   # cut
    t = list(b)   # ingre
    for i in range(len(c)):
        if c[i] != 0:
            t_cut[c[i],t[i]] = 1
    return t_cut

In [14]:
# Get all recipes
All_recipe = []
for p in IngreLabel:
    ingre = [int(i) for i in p[1:]]
    r = recipe_retrival(ingre)
    All_recipe.append(r)
# R = torch.from_numpy(np.asarray(R))

In [15]:
# Get dish name for all recipes
Dish_name = []
for p in IngreLabel:
    tmp = p[0]
    tmp = tmp.split('/')
    Dish_name.append(int(tmp[1]))

In [16]:
#calculate cross entropy between image representation p and target representation t

def loss_function(p_ingre, p_cut, t_ingre, t_cut):
  
    # loss of ingredients
    L1 = nn.BCEWithLogitsLoss(p_ingre, t_ingre)
  
    # ground-truth ingredient set
    a = t_ingre.nonzero().view(-1)
  
    # conditional cutting tensor
    new_p_cut = torch.zeros(p_cut.size())
    for j in a.numpy():
        for i in range(p_cut.size(0)):
            new_p_cut[i][j] = p_cut[i][j]
    # loss of cutting
    L2 = nn.BCEWithLogitsLoss(new_p_cut, t_cut)
  
    L = L1 + L2
    return L

In [17]:
# recipe R is represented by a list of list(ingredient, cutting, cooking)
# def similarity(Q_ingre, Q_cut, Q_cook, R):
#     simi = 0
#     for triplet in R:
#         ing, cut, cook = triplet[0], triplet[1], triplet[2]
#     simi += Q_ingre[ing] + cc_rate * (Q_cut[cut][ing] + Q_cook[cook][ing])
#     simi /= len(R)
  
#     return simi

def similarity(Q_ingre, Q_cut, R):
    simi = 0
    for doublet in R:
        ing, cut = doublet[0], boublet[1]
    simi += Q_ingre[ing] + cc_rate * Q_cut[cut][ing]
    simi /= len(R)
  
    return simi

def pred_dish(Q_ingre,Q_cut,all_recipe):
    simi = 0
    res = -1
    for idx, recipe in enumerate(all_recipe):
        tmp = similarity(Q_ingre, Q_cut, recipe)
        if tmp > simi:
            simi = tmp
            res = idx
    return(Dish_name[res])

## Training

### Calculating preconvoluted features

In [18]:
def preconvfeat(dataset):
    conv_features = []
    labels_list = []
    paths_list = []
    for data in dataset:
        inputs,labels, paths = data
#         print(paths)
        if use_gpu:
            inputs , labels = Variable(inputs.cuda()),Variable(labels.cuda())
        else:
            inputs , labels = Variable(inputs),Variable(labels)
        model = gpu(vggfeature())
        x = model.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
        paths_list.extend(list(paths))
    conv_features = np.concatenate([[feat] for feat in conv_features])
    return (conv_features,labels_list,paths_list)

In [19]:
dataset_valid = torch.utils.data.DataLoader(dsets['val'], batch_size=5, shuffle=True, num_workers=6)
count = 1
for data in dataset_valid:
    if count == 1:
        inputs_try,labels_try, paths_try = data
print(labels_try)
inputs_try , labels_try = gpu(inputs_try),gpu(labels_try)
model = gpu(vggfeature())
x_try = model.features(inputs_try)
print(x_try)

tensor([88])
tensor([[[[ 0.1169,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  1.6182,  2.1719,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.1212,  1.5126,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.7239,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 4.3125,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.2739],
          [ 0.2786,  3.2627,  2.9672,  ...,  3.4887,  0.9362,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.568

In [21]:
%%time
conv_feat_train,labels_train, paths_train = preconvfeat(dset_loaders['train'])

CPU times: user 29min 37s, sys: 9min 18s, total: 38min 55s
Wall time: 38min 55s


In [22]:
%%time
conv_feat_val,labels_val, paths_val = preconvfeat(dset_loaders['val'])

CPU times: user 5min, sys: 1min 8s, total: 6min 8s
Wall time: 6min 9s


In [23]:
# %%time
# conv_feat_test, labels_test, paths_test = preconvfeat(dset_loaders['test'])

In [19]:
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]
    
# %mkdir ./cross_model

In [25]:
save_array(os.path.join('./cross_model','feat_train.bc'),conv_feat_train)
save_array(os.path.join('./cross_model','labels_train.bc'),labels_train)
save_array(os.path.join('./cross_model','paths_train.bc'),paths_train)
save_array(os.path.join('./cross_model','feat_val.bc'),conv_feat_val)
save_array(os.path.join('./cross_model','labels_val.bc'),labels_val)
save_array(os.path.join('./cross_model','paths_val.bc'),paths_val)
# save_array(os.path.join('./cross_model','feat_test.bc'),conv_feat_test)
# save_array(os.path.join('./cross_model','labels_test.bc'),labels_test)

### Loading Preconvoluted features

In [20]:
conv_feat_train = load_array('./cross_model/feat_train.bc')
labels_train = load_array('./cross_model/labels_train.bc')
paths_train = load_array('./cross_model/paths_train.bc')
conv_feat_val = load_array('./cross_model/feat_val.bc')
labels_val = load_array('./cross_model/labels_val.bc')
paths_val = load_array('./cross_model/paths_val.bc')
# conv_feat_test = load_array(data_dir+'/cross_model/conv_feat_test.bc')
# labels_test = load_array(data_dir+'/cross_model/labels_test.bc')

In [21]:
conv_feat_train.shape

(66071, 512, 7, 7)

In [22]:
def data_gen(conv_feat,labels,paths,batch_size=64,shuffle=True):
    labels = np.array(labels)
    paths = np.array(paths)
    if shuffle:
        index = np.random.permutation(len(conv_feat))
#         print(index)
#         print(conv_feat)
        conv_feat = conv_feat[index]
        labels = labels[index]
        paths = paths[index]
    for idx in range(0,len(conv_feat),batch_size):
        yield(conv_feat[idx:idx+batch_size],labels[idx:idx+batch_size], paths[idx:idx+batch_size])

In [45]:
# n_iter=10
batch_size=50
learning_rate=1e-2

optimizer = optim.SGD(embedding().parameters(), learning_rate)

def train_model_emb(model,size_train,size_val,conv_feat_train=None,labels_train=None, paths_train=None, 
                conv_feat_val=None,labels_val=None,paths_val=None,epochs=1,optimizer=None,train=True,shuffle=True):
# def train_model_emb(model,size_train,dataloader=None,epochs=1,optimizer=None,train=True,shuffle=True):
    if train:
        model.train()
    else:
        model.eval()
#     gpu(model)    
    LOSS = []
    ACC = []
    LOSS_V = []
    ACC_V = []
#     train_loader = dataloader['train']
    
    for epoch in range(epochs):
        batches = data_gen(conv_feat=conv_feat_train,labels=labels_train,paths=paths_train,shuffle=shuffle)
#         print(batches)
        total = 0
        running_loss = 0.0
        running_corrects = 0
#         for inputs,classes,paths in train_loader:
        for inputs,classes,paths in batches:
#             print(type(inputs[0]))
#             print(type(classes[0]))
#             print(type(paths[0]))
            inputs,classes = gpu(torch.from_numpy(inputs)),gpu(torch.from_numpy(classes))   
#             inputs = inputs.view(inputs.size(0), -1)
            print(inputs.size())
#             inputs = gpu(inputs).view(inputs.size(0), 3, 224, 224)
    
            fea_embeddings, ing_embeddings, cut_embeddings  = model(inputs)
            loss = []
            preds = []
            for i in range(fea_embeddings.size(0)):
                fea_emb = fea_embeddings[i]
                ing_emb = ing_embeddings[i]
                cut_emb = cut_embeddings[i]
                p_ingre,p_cut = pooling(ing_emb, cut_emb)
                t_ingre = Get_ingre(paths[i])
                t_cut = Get_cut(t_ingre)
                # loss
                l = loss_function(p_ingre, p_cut, t_ingre, t_cut)
                # similarity, retrival
#                 R = recipe_retrival(t_ingre)
#                 simi = similarity(p_ingre, p_cut, R)
                pred = pred_dish(p_ingre,p_cut,All_recipe)
                loss.append(l)
                preds.append(pred)
            loss = torch.FloatTensor(loss)
            preds = torch.FloatTensor(preds)
            
            if optimizer is None:
                raise ValueError('Pass optimizer for train mode')
            optimizer = optimizer
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
#             if train:
#                 if optimizer is None:
#                     raise ValueError('Pass optimizer for train mode')
#                 optimizer = optimizer
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()            
            
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / size_train
        epoch_acc = running_corrects.data.item() / size_train
        LOSS.append(epoch_loss)
        ACC.append(epoch_acc)
        print('Epoch: {} Training Loss: {:.4f} Training Acc: {:.4f}'.format(
                     epoch+1, epoch_loss, epoch_acc))
        
#         batches = data_gen(conv_feat=conv_feat_val,labels=labels_val,paths=paths_val,shuffle=shuffle)
#         total = 0
#         running_loss = 0.0
#         running_corrects = 0
#         for inputs,classes,paths in batches:
#             inputs,classes = gpu(torch.from_numpy(inputs)),gpu(torch.from_numpy(classes))       
                            
#             inputs = inputs.view(inputs.size(0), -1)
            
#             fea_embeddings, ing_embeddings, cut_embeddings  = model(inputs)
#             loss = []
#             preds = []
#             for i in range(fea_embeddings.size(0)):
#                 fea_emb = fea_embeddings[i]
#                 ing_emb = ing_embeddings[i]
#                 cut_emb = cut_embeddings[i]
#                 p_ingre,p_cut = pooling(ing_emb, cut_emb)
#                 t_ingre = Get_ingre(paths[i])
#                 t_cut = Get_cut(t_ingre)
#                 # loss
#                 l = loss_function(p_ingre, p_cut, t_ingre, t_cut)
#                 # similarity, retrival           
#                 pred = pred_dish(p_ingre,p_cut,All_recipe)
#                 loss.append(l)
#                 preds.append(pred)
#             loss = torch.FloatTensor(loss)
#             preds = torch.FloatTensor(preds)
#             # statistics
#             running_loss += loss.data.item()
#             running_corrects += torch.sum(preds == classes.data)
#         epoch_loss = running_loss / size_val
#         epoch_acc = running_corrects.data.item() / size_val
#         LOSS_V.append(epoch_loss)
#         ACC_V.append(epoch_acc)
#         print('Epoch: {} Validation Loss: {:.4f} Validation Acc: {:.4f}'.format(
#                      epoch+1, epoch_loss, epoch_acc))
        
#     return 
#     return LOSS, ACC, LOSS_V, ACC_V
    return LOSS, ACC

In [48]:
LOSS, ACC = (train_model_emb(model=gpu(embedding()),
            size_train=dset_sizes['train'],size_val=dset_sizes['val'],
            conv_feat_train=conv_feat_train,labels_train=labels_train,
            paths_train=paths_train,
            conv_feat_val=conv_feat_val,labels_val=labels_val,
            paths_val=paths_val,
            epochs=50,optimizer=optimizer,train=True,shuffle=True))

RuntimeError: CUDA error: device-side assert triggered

In [44]:
# n_iter=10
batch_size=50
learning_rate=1e-2

optimizer = optim.SGD(vggfeature().parameters(), learning_rate)

# def train_model(model,size_train,size_val,conv_feat_train=None,labels_train=None, paths_train=None, 
#                 conv_feat_val=None,labels_val=None,paths_val=None,epochs=1,optimizer=None,train=True,shuffle=True):
def train_model(model,size_train,dataloader=None,epochs=1,optimizer=None,train=True,shuffle=True):
    if train:
        model.train()
    else:
        model.eval()
#     gpu(model)    
    LOSS = []
    ACC = []
    LOSS_V = []
    ACC_V = []
    train_loader = dataloader['train']
    
    for epoch in range(epochs):
#         batches = data_gen(conv_feat=conv_feat_train,labels=labels_train,paths=paths_train,shuffle=shuffle)
#         print(batches)
        total = 0
        running_loss = 0.0
        running_corrects = 0
        for inputs,classes,paths in train_loader:
#         for inputs,classes,paths in batches:
#             print(type(inputs[0]))
#             print(type(classes[0]))
#             print(type(paths[0]))
#             inputs,classes = gpu(torch.from_numpy(inputs)),gpu(torch.from_numpy(classes))        
#             inputs = inputs.view(inputs.size(0), -1)
#             print(inputs.size())
            inputs = gpu(inputs).view(inputs.size(0), 3, 224, 224)
    
            fea_embeddings, ing_embeddings, cut_embeddings  = model(inputs)
            loss = []
            preds = []
            for i in range(fea_embeddings.size(0)):
                fea_emb = fea_embeddings[i]
                ing_emb = ing_embeddings[i]
                cut_emb = cut_embeddings[i]
                p_ingre,p_cut = pooling(ing_emb, cut_emb)
                t_ingre = Get_ingre(paths[i])
                t_cut = Get_cut(t_ingre)
                # loss
                l = loss_function(p_ingre, p_cut, t_ingre, t_cut)
                # similarity, retrival
#                 R = recipe_retrival(t_ingre)
#                 simi = similarity(p_ingre, p_cut, R)
                pred = pred_dish(p_ingre,p_cut,All_recipe)
                loss.append(l)
                preds.append(pred)
            loss = torch.FloatTensor(loss)
            preds = torch.FloatTensor(preds)
            
            if optimizer is None:
                raise ValueError('Pass optimizer for train mode')
            optimizer = optimizer
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
#             if train:
#                 if optimizer is None:
#                     raise ValueError('Pass optimizer for train mode')
#                 optimizer = optimizer
#                 optimizer.zero_grad()
#                 loss.backward()
#                 optimizer.step()            
            
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / size_train
        epoch_acc = running_corrects.data.item() / size_train
        LOSS.append(epoch_loss)
        ACC.append(epoch_acc)
        print('Epoch: {} Training Loss: {:.4f} Training Acc: {:.4f}'.format(
                     epoch+1, epoch_loss, epoch_acc))
        
#         batches = data_gen(conv_feat=conv_feat_val,labels=labels_val,paths=paths_val,shuffle=shuffle)
#         total = 0
#         running_loss = 0.0
#         running_corrects = 0
#         for inputs,classes,paths in batches:
#             inputs,classes = gpu(torch.from_numpy(inputs)),gpu(torch.from_numpy(classes))       
                            
#             inputs = inputs.view(inputs.size(0), -1)
            
#             fea_embeddings, ing_embeddings, cut_embeddings  = model(inputs)
#             loss = []
#             preds = []
#             for i in range(fea_embeddings.size(0)):
#                 fea_emb = fea_embeddings[i]
#                 ing_emb = ing_embeddings[i]
#                 cut_emb = cut_embeddings[i]
#                 p_ingre,p_cut = pooling(ing_emb, cut_emb)
#                 t_ingre = Get_ingre(paths[i])
#                 t_cut = Get_cut(t_ingre)
#                 # loss
#                 l = loss_function(p_ingre, p_cut, t_ingre, t_cut)
#                 # similarity, retrival           
#                 pred = pred_dish(p_ingre,p_cut,All_recipe)
#                 loss.append(l)
#                 preds.append(pred)
#             loss = torch.FloatTensor(loss)
#             preds = torch.FloatTensor(preds)
#             # statistics
#             running_loss += loss.data.item()
#             running_corrects += torch.sum(preds == classes.data)
#         epoch_loss = running_loss / size_val
#         epoch_acc = running_corrects.data.item() / size_val
#         LOSS_V.append(epoch_loss)
#         ACC_V.append(epoch_acc)
#         print('Epoch: {} Validation Loss: {:.4f} Validation Acc: {:.4f}'.format(
#                      epoch+1, epoch_loss, epoch_acc))
        
#     return 
#     return LOSS, ACC, LOSS_V, ACC_V
    return LOSS, ACC


NameError: name 'vggfeature' is not defined

RuntimeError: CUDA error: device-side assert triggered

In [27]:
LOSS, ACC = train_model(model=gpu(vggfeature()),size_train=dset_sizes['train'],
                        dataloader=dset_loaders,epochs=50,optimizer=optimizer,train=True,shuffle=True)

RuntimeError: CUDA error: device-side assert triggered

In [101]:
%%time
LOSS, ACC, LOSS_V, ACC_V = (train_model(model=gpu(vggfeature()),
            size_train=dset_sizes['train'],size_val=dset_sizes['val'],
            conv_feat_train=conv_feat_train,labels_train=labels_train,
            paths_train=paths_train,
            conv_feat_val=conv_feat_val,labels_val=labels_val,
            paths_val=paths_val,
            epochs=50,optimizer=optimizer,train=True,shuffle=True))

torch.Size([64, 512, 7, 7])


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[64, 512, 7, 7] to have 3 channels, but got 512 channels instead

In [35]:
model = vggfeature()
print(model)

vggfeature(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17):